In [1]:
words = open ('names.txt','r').read().splitlines()

In [2]:
a = b = 3

In [ ]:
open ('names.txt','r').read().splitlines()

In [ ]:
words[:10]

In [ ]:
len(words)

In [ ]:
min(len(w) for w in words)

In [ ]:
max(len(w) for w in words)

In [8]:
b={}
for w in words:
    chs=['<S>']+ list(w) +['<E>']
    for ch1,ch2 in zip(chs,chs[1:]):
        biagram=(ch1,ch2)
        #print(biagram)
        b[biagram]=b.get(biagram,0)+1

In [ ]:
sorted(b.items(),key=lambda kv: -kv[1])

In [ ]:
import torch

In [11]:
N=torch.zeros((27,27),dtype=torch.int32)

In [ ]:
chars=sorted(list(set(''.join(words))))
stoi={s:i+1 for i,s in enumerate(chars)}  #a:0 b:1
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

print(stoi)

In [13]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1,ix2] += 1

In [ ]:
p=N[0].float()
p=p/p.sum()
p

In [ ]:
N[0]

In [ ]:
g=torch.Generator().manual_seed(2147483647)
p=torch.rand(3,generator=g)
p=p/p.sum()
torch.multinomial(p,num_samples=100,replacement=True,generator=g)

In [17]:
P=(N+1).float()
P=P/P.sum(1,keepdim=True) #  分母 27*1

In [ ]:
g=torch.Generator().manual_seed(2147483647)
for i in range(5):

    out=[]
    ix=0
    while True:
        p = P[ix]
        #p = p / p.sum()
        ix=torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
        out.append(itos[ix])
        if ix==0:
            break
    print(''.join(out))

In [ ]:
log_likelihood=0.0
n=0

for w in words:
    chs=['.']+ list(w) +['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob=P[ix1,ix2]
        logprob=torch.log(prob)
        log_likelihood += logprob
        n += 1

nll=-log_likelihood
nll=nll/n
print(f'{nll}')
print(f'{log_likelihood}')
#全部的是错的

In [ ]:
xs,ys = [],[]

for w in words:
    chs=['.']+ list(w) +['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        print(ch1,ch2)
        xs.append(ix1)
        ys.append(ix2)

xs=torch.tensor(xs)
num=xs.nelement()
ys=torch.tensor(ys)
g=torch.Generator().manual_seed(2147483647)
W=torch.rand((27,27),generator=g,requires_grad=True)

In [ ]:
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch
%matplotlib inline
for k in range(100):
    xenc=F.one_hot(xs,num_classes=27).float()
    #plt.imshow(xenc)  ？？？错了
    logits=xenc @ W
    counts=logits.exp()
    probs=counts/counts.sum(1,keepdim=True)
    loss=-probs[torch.arange(num),ys].log().mean()
    print(loss.item())

    W.grad=None
    loss.backward()

    W.data += -100 * W.grad

    

In [ ]:
g=torch.Generator().manual_seed(2147483647)
for i in range(50):

    out=[]
    ix=0
    while True:
        xenc=F.one_hot(torch.tensor(ix),num_classes=27).float()
        logits=xenc @ W
        counts=logits.exp()
        probs=counts/counts.sum(0,keepdim=True)
        ix=torch.multinomial(probs,num_samples=1,replacement=True,generator=g).item()
        out.append(itos[ix])
        if ix==0:
            break
    print(''.join(out))

In [ ]:
W[:3]

In [24]:
xenc=F.one_hot(xs,num_classes=27).float()

In [25]:
x = xenc[0:5]

In [ ]:
x

In [ ]:
x.sum(0)